### GFGGAN

In [1]:
import cv2
import torch
import numpy as np
import imutils
import pickle
import time
from gfpgan import GFPGANer

# === Paths ===
embeddingModel = r"D:\PythonProject\Face_Recognition_DL\openface.nn4.small2.v1.t7"
recognizerFile = r"D:\PythonProject\Face_Recognition_DL\Output_Models\recognizer_Openface01.pickle"
labelEncFile = r"D:\PythonProject\Face_Recognition_DL\Output_Models\LE_Openface01.pickle"
prototxt_path = r"D:\PythonProject\Face_Recognition_DL\model\deploy.prototxt.txt"
model_path = r"D:\PythonProject\Face_Recognition_DL\model\res10_300x300_ssd_iter_140000.caffemodel"
gfpgan_model_path = r"D:\PythonProject\Face_Recognition_DL\model\GFPGANv1.4.pth"

# === Configuration ===
face_conf_threshold = 0.5
recognition_conf_threshold = 0.5

# === Load models ===
print("[INFO] Loading face detector...")
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

print("[INFO] Loading face embedder...")
embedder = cv2.dnn.readNetFromTorch(embeddingModel)

print("[INFO] Loading recognizer and label encoder...")
recognizer = pickle.loads(open(recognizerFile, "rb").read())
le = pickle.loads(open(labelEncFile, "rb").read())

print("[INFO] Loading GFPGAN model...")
gfpgan = GFPGANer(
    model_path=gfpgan_model_path,
    upscale=1,
    arch='clean',
    channel_multiplier=2,
    bg_upsampler=None
)

# === Start webcam ===
print("[INFO] Starting video stream...")
cam = cv2.VideoCapture(0)
time.sleep(2.0)

while True:
    ret, frame = cam.read()
    if not ret:
        break

    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]

    # Detect faces using DNN
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                                 (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence < face_conf_threshold:
            continue

        # Get face bounding box
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        face = frame[startY:endY, startX:endX]
        (fH, fW) = face.shape[:2]

        if fW < 20 or fH < 20:
            continue

        # === Enhance face using GFPGAN ===
        try:
            face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            _, restored_faces, _ = gfpgan.enhance(
                face_rgb, has_aligned=False, only_center_face=True, paste_back=False
            )
            enhanced_face = cv2.cvtColor(restored_faces[0], cv2.COLOR_RGB2BGR)
        except Exception as e:
            print(f"[WARNING] GFPGAN enhancement failed: {e}")
            enhanced_face = face

        # === Face recognition ===
        faceBlob = cv2.dnn.blobFromImage(enhanced_face, 1.0 / 255, (96, 96),
                                         (0, 0, 0), swapRB=True, crop=False)
        embedder.setInput(faceBlob)
        vec = embedder.forward()

        preds = recognizer.predict_proba([vec.flatten()])[0]
        j = np.argmax(preds)
        proba = preds[j]
        name = le.classes_[j] if proba >= recognition_conf_threshold else "Unknown"

        # === Draw results ===
        text = f"{name} : {proba * 100:.2f}%"
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 1)
        cv2.putText(frame, text, (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 1)

        # Print recognition info
        if name != "Unknown":
            print(f"[INFO] Recognized: {name}, Probability: {proba * 100:.2f}%")
        else:
            print(f"[INFO] Unknown face detected with probability: {proba * 100:.2f}%")

    cv2.imshow("Frame", frame)

    if cv2.getWindowProperty("Frame", cv2.WND_PROP_VISIBLE) < 1:
        break

    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC
        break

# === Cleanup ===
cam.release()
cv2.destroyAllWindows()


d:\PythonProject\Face_Recognition_DL\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Loading face detector...
[INFO] Loading face embedder...
[INFO] Loading recognizer and label encoder...


d:\PythonProject\Face_Recognition_DL\.venv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\PythonProject\Face_Recognition_DL\.venv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[INFO] Loading GFPGAN model...


d:\PythonProject\Face_Recognition_DL\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\PythonProject\Face_Recognition_DL\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[INFO] Starting video stream...
[INFO] Recognized: Srijani_Test_Dataset, Probability: 100.00%
[INFO] Recognized: Agnishwar_Das, Probability: 90.14%
[INFO] Recognized: Srijani_Test_Dataset, Probability: 66.82%
[INFO] Recognized: Srijani_Test_Dataset, Probability: 80.10%
[INFO] Recognized: Agnishwar_Das, Probability: 93.38%
[INFO] Recognized: Srijani_Test_Dataset, Probability: 73.55%
[INFO] Recognized: Srijani_Test_Dataset, Probability: 84.88%
[INFO] Recognized: Agnishwar_Das, Probability: 62.06%
[INFO] Recognized: Srijani_Test_Dataset, Probability: 93.62%


### ERSGAN

In [2]:
import cv2
import torch
import numpy as np
import imutils
import pickle
import time
from realesrgan import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet

# === Paths ===
embeddingModel = r"D:\PythonProject\Face_Recognition_DL\openface.nn4.small2.v1.t7"
recognizerFile = r"D:\PythonProject\Face_Recognition_DL\Output_Models\recognizer_Openface01.pickle"
labelEncFile = r"D:\PythonProject\Face_Recognition_DL\Output_Models\LE_Openface01.pickle"
prototxt_path = r"D:\PythonProject\Face_Recognition_DL\model\deploy.prototxt.txt"
model_path = r"D:\PythonProject\Face_Recognition_DL\model\res10_300x300_ssd_iter_140000.caffemodel"
esrgan_model_path = r"D:\PythonProject\Face_Recognition_DL\model\RealESRGAN_x4plus.pth"

# === Configuration ===
face_conf_threshold = 0.5
recognition_conf_threshold = 0.5

# === Load face detector ===
print("[INFO] Loading face detector...")
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

# === Load face embedder ===
print("[INFO] Loading face embedder...")
embedder = cv2.dnn.readNetFromTorch(embeddingModel)

# === Load recognizer and label encoder ===
print("[INFO] Loading recognizer and label encoder...")
recognizer = pickle.loads(open(recognizerFile, "rb").read())
le = pickle.loads(open(labelEncFile, "rb").read())

# === Load Real-ESRGAN model ===
print("[INFO] Loading Real-ESRGAN model...")
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                num_block=23, num_grow_ch=32, scale=4)

upscaler = RealESRGANer(
    scale=4,
    model_path=esrgan_model_path,
    model=model,
    tile=128,         # Set to 128 for better performance, can try 64 for faster results
    tile_pad=10,
    pre_pad=0,
    half=False,       # True if using GPU with FP16 support
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

# === Start video stream ===
print("[INFO] Starting video stream...")
cam = cv2.VideoCapture(0)
time.sleep(2.0)

while True:
    ret, frame = cam.read()
    if not ret:
        break

    # Resize frame for processing efficiency
    frame_resized = imutils.resize(frame, width=600)
    (h, w) = frame_resized.shape[:2]

    # Face detection
    blob = cv2.dnn.blobFromImage(cv2.resize(frame_resized, (300, 300)), 1.0,
                                 (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence < face_conf_threshold:
            continue

        # Bounding box
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        face = frame_resized[startY:endY, startX:endX]
        (fH, fW) = face.shape[:2]

        if fW < 20 or fH < 20:
            continue

        # === Enhance face using Real-ESRGAN ===
        try:
            face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            enhanced_rgb, _ = upscaler.enhance(face_rgb, outscale=1)
            enhanced_face = cv2.cvtColor(enhanced_rgb, cv2.COLOR_RGB2BGR)
        except Exception as e:
            print(f"[WARNING] Real-ESRGAN enhancement failed: {e}")
            enhanced_face = face

        # === Face embedding and recognition ===
        faceBlob = cv2.dnn.blobFromImage(enhanced_face, 1.0 / 255, (96, 96),
                                         (0, 0, 0), swapRB=True, crop=False)
        embedder.setInput(faceBlob)
        vec = embedder.forward()

        preds = recognizer.predict_proba([vec.flatten()])[0]
        j = np.argmax(preds)
        proba = preds[j]
        name = le.classes_[j] if proba >= recognition_conf_threshold else "Unknown"

        # === Draw results ===
        text = f"{name} : {proba * 100:.2f}%"
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame_resized, (startX, startY), (endX, endY), (0, 0, 255), 1)
        cv2.putText(frame_resized, text, (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 1)

        # Console log
        if name != "Unknown":
            print(f"[INFO] Recognized: {name}, Probability: {proba * 100:.2f}%")
        else:
            print(f"[INFO] Unknown face detected with probability: {proba * 100:.2f}%")

    # Display the processed frame with detected faces
    cv2.imshow("Frame", frame_resized)

    # Stop the loop if the user presses 'ESC'
    if cv2.getWindowProperty("Frame", cv2.WND_PROP_VISIBLE) < 1:
        break

    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC key to exit
        break

# === Cleanup ===
cam.release()
cv2.destroyAllWindows()


[INFO] Loading face detector...
[INFO] Loading face embedder...
[INFO] Loading recognizer and label encoder...
[INFO] Loading Real-ESRGAN model...


d:\PythonProject\Face_Recognition_DL\.venv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\PythonProject\Face_Recognition_DL\.venv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[INFO] Starting video stream...
	Tile 1/6
	Tile 2/6
	Tile 3/6
	Tile 4/6
	Tile 5/6
	Tile 6/6
[INFO] Recognized: Srijani_Test_Dataset, Probability: 100.00%


In [3]:
pip install realesrgan basicsr

  Using cached realesrgan-0.3.0-py3-none-any.whl.metadata (17 kB)
Using cached realesrgan-0.3.0-py3-none-any.whl (26 kB)
Note: you may need to restart the kernel to use updated packages.
